# Concise guide

> LLM-friendly opinionated sample FastHTML code with MonsterUI and Fastlite

- order: 3

In [ ]:
from fasthtml.common import *

## A Minimal Application

A minimal FastHTML application looks something like this:

```python
# This is a meta-package which includes all of the key symbols from FastHTML and Starlette 
from fasthtml.common import *
# This returns the FastHTML application object as well as a shortcut to `app.route`
app,rt = fast_app()
# Passing a path to `rt` is optional. If not passed, the function name is the route ('/foo') here.
# By default, both GET and POST HTTP methods are handled.
# Type annotated params are passed as query params unless a path param is defined (which it isn't here)
@rt
def foo(name:str):
    # `Title` and `P` here are "fasttags". Fasttags are direct m-expression mappings of HTML tags to Python functions with positional and named parameters. All standard HTML tags are included in the common wildcard import.
    # When a tuple is returned, this returns concatenated HTML partials. HTMX by default will use a title HTML partial to set the current page name. 
    # FastHTML will automatically return a complete HTML document with appropriate headers if a normal HTTP request is received. For an HTMX request, however, just the partials are returned.
    return Title("FastHTML"), H1("My web app"), P(f"Let's do this, {name}!")
# By default with `serve` uvicorn hosts on port 5001. `if __name__` etc is *not* needed since `serve` has this check internally.
serve()
```

To run this web app:

```bash
python main.py  # access via localhost:5001
```

fasttags can be considered as m-expressions, with some simple sugar added. Positional parameters map to children, and named parameters map to attributes. Aliases must be used for Python reserved words. 

In [ ]:
tags = Title("FastHTML"), H1("My web app"), P(f"Let's do this!", cls="myclass")
tags

(title(('FastHTML',),{}),
 h1(('My web app',),{}),
 p(("Let's do this!",),{'class': 'myclass'}))

This example demonstrates key aspects of how FastHTML components handle attributes:

```python
Label(
    "Choose an option", 
    Select(
        Option("one", value="1", selected=True),  # True renders just the attribute name
        Option("two", value=2, selected=False),   # Non-string values are converted to strings. False omits the attribute entirely
        cls="selector", id="counter",             # 'cls' becomes 'class'
        **{'@click':"alert('Clicked');"},         # Dict unpacking for attributes with special characters
    ),
    _for="counter",                               # '_for' becomes 'for' (can also use 'fr')
)
```

You can create new FT components by importing the new component from `fasthtml.components`. If the component doesn't exist within that module, FastHTML will create it.

In [ ]:
from fasthtml.components import Some_never_before_used_tag

Some_never_before_used_tag()

```html
<some-never-before-used-tag></some-never-before-used-tag>
```

When creating a response,  FastHTML will automatically render tags using the `to_xml` function.

In [ ]:
to_xml(tags)

'<title>FastHTML</title>\n<h1>My web app</h1>\n<p class="myclass">Let&#x27;s do this!</p>\n'

The `Script` function allows you to include JavaScript. You can use Python to generate parts of your JS or JSON like this:

```python
# In future snippets this import will not be shown, but is required
from fasthtml.common import * 
app,rt = fast_app(hdrs=[Script(src="https://cdn.plot.ly/plotly-2.32.0.min.js")])
# `index` is a special function name which maps to the `/` route. 
@rt
def index():
    data = {'somedata':'fill me in…'}
    # `Titled` is a function which returns a title tag and an h1 tag with the 1st param, along with all other params as HTML in a `Main` parent element
    return Titled("Chart Demo", Div(id="myDiv"), Script(f"var data = {data}; Plotly.newPlot('myDiv', data);"))
# In future snippets `serve() will not be shown, but is required
serve()
```

Route details can be specified as decorator parameters.

```python
# In future snippets we'll skip showing the `fast_app` call if it has no params
app, rt = fast_app(
    pico=False, # The Pico CSS framework is included by default, so pass `false` to disable it if needed.
    # These are added to the `head` part of the page for non-HTMX requests.
    hdrs=(
        Link(rel='stylesheet', href='assets/normalize.min.css', type='text/css'),
        Link(rel='stylesheet', href='assets/sakura.css', type='text/css'),
        Style("p {color: red;}",
        # by default, all standard static extensions are served statically from the web app dir, which can be modified using `static_path`
        static_path='public')
)
# Path parameter parsing is handled by Starlette, so all of the same functionality and syntax is supported.
# However, it is more idiomatic to use query parameters in FastHTML where possible.
@rt("/hello/{name}", method="GET")
def somefunc(name:str): return Titled(f"Hello, {name}!")
```

Instead of returning a FastTag or tuple of FastTags, standard Starlette response types can also be returned in which case they are used directly in the same way that Starlette does. 

```python
@rt("/{fname:path}.{ext:static}")
async def get(fname:str, ext:str):  return FileResponse(f'public/{fname}.{ext}')
```

```python
# `MarkdownJS` and `HighlightJS` are available via concise functions
hdrs = (MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']), )
app, rt = fast_app(hdrs=hdrs)
@rt
def index(req): return Titled("Markdown rendering example",
                              # This will be markdown rendered to HTML
                              Div("*hi* there",cls="marked"),
                              # This will be syntax highlighted
                              Pre(Code("def foo(): pass")))
serve()
```

Returning a JSON serializable dict from a handler will cause the handler to return JSON. 

`ft` components ("fasttags") can be combined by defining them as a function.

In [ ]:
def Hero(title, statement): return Div(H1(title),P(statement), cls="hero")
to_xml(Hero("Hello World", "This is a hero statement"))

'<div class="hero">\n  <h1>Hello World</h1>\n  <p>This is a hero statement</p>\n</div>\n'

Classes with `__ft__` defined are rendered using that method.

In [ ]:
from dataclasses import dataclass

class FtTest:
    def __ft__(self): return P('test')
    
to_xml(FtTest())

'<p>test</p>\n'

In [ ]:
from dataclasses import dataclass

In [ ]:
app,rt = fast_app()

When a route handler function is used as a fasttag attribute (such as `href`, `hx_get`, or `action`) it is converted to that route's path. `fill_form` is used to copy an object's matching attrs into matching-name form fields.

In [ ]:
@dataclass
class Profile: email:str; phone:str; age:int
email = 'john@example.com'
profiles = {email: Profile(email=email, phone='123456789', age=5)}
@rt
def profile(email:str): return fill_form(profile_form, profiles[email])

profile_form = Form(method="post", action=profile)(
        Fieldset(
            Label('Email', Input(name="email")),
            Label("Phone", Input(name="phone")),
            Label("Age", Input(name="age"))),
        Button("Save", type="submit"))

A `to` method is added to route functions, which returns a route path with params as query params.

In [ ]:
path = profile.to(email="john@example.com")
path

'/profile?email=john%40example.com'

The `url_for` method of the request object can be used to generate URLs for named routes. It takes the route name as its first argument, followed by any path parameters needed for that route. Useful for generating absolute URLs, ensuring that the correct host and scheme are included, even if the application is accessed through different domains or protocols.

The `url_path_for` method of the application can be used to generate URL paths for named routes. Unlike `url_for`, it returns only the path component of the URL, without the scheme or host.

We can use `TestClient` for testing.

In [ ]:
from starlette.testclient import TestClient

In [ ]:
client = TestClient(app)
htmx_req = {'HX-Request':'1'}
print(client.get(path, headers=htmx_req).text)

<form enctype="multipart/form-data" method="post" action="/profile"><fieldset><label>Email       <input name="email" value="john@example.com">
</label><label>Phone       <input name="phone" value="123456789">
</label><label>Age       <input name="age" value="5">
</label></fieldset><button type="submit">Save</button></form>


When a dataclass, namedtuple, etc. is used as a type annotation, the form body will be unpacked into matching attribute names automatically. 

In [ ]:
@rt
def edit_profile(profile: Profile):
    profiles[email]=profile
    return RedirectResponse(url=path)

new_data = dict(email='john@example.com', phone='7654321', age=25)
print(client.post("/edit_profile", data=new_data, headers=htmx_req).text)

<form enctype="multipart/form-data" method="post" action="/profile"><fieldset><label>Email       <input name="email" value="john@example.com">
</label><label>Phone       <input name="phone" value="7654321">
</label><label>Age       <input name="age" value="25">
</label></fieldset><button type="submit">Save</button></form>


The general rules for rendering are: 
- `__ft__` method will be called (for default components like `P`, `H2`, etc. or if you define your own components)
- If you pass a string, it will be escaped
- On other python objects, `str()` will be called

As a consequence, if you want to include plain HTML tags directly into e.g. a `Div()` they will get escaped by default (as a security measure to avoid code injections). This can be avoided by using `Safe(...)`, e.g to show a data frame use `Div(NotStr(df.to_html()))`.

FastHTML allows customization of exception handlers.

```python
def not_found(req, exc): return Titled("404: I don't exist!")
exception_handlers = {404: not_found}
app, rt = fast_app(exception_handlers=exception_handlers)
```

We can set cookies using the `cookie()` function.

In [ ]:
@rt
def setcook(): return P(f'Setting a cookie'), cookie('mycookie', 'foobar')
print(client.get('/setcook', headers=htmx_req).text)

 <p>Setting a cookie</p>



In [ ]:
@rt
def getcook(mycookie:str): return f'Cookie was {mycookie}'
# Note that if handlers return text, instead of fasttags, then a plain text response is automatically created
print(client.get('/getcook').text)

Cookie was foobar


FastHTML provide access to Starlette's request object automatically using special `request` parameter name (or any prefix of that name).

In [ ]:
@rt
def headers(req): return req.headers['host']

FastHTML provides access to Starlette's session middleware automatically using the special `session` parameter name (or any prefix of that name).

Enums can be used to constrain the values accepted for a route parameter.

In [ ]:
names = str_enum('names', 'Alice', 'Bev', 'Charlie')

In [ ]:
@rt
def adder(sess, nm: names):
    sess.setdefault('attendees', '')
    sess['attendees'] = sess.get('attendees') + f', {nm}'
    return Response(f'The attentees are {sess["attendees"]}.')

Handler functions can return the `HtmxResponseHeaders` object to set HTMX-specific response headers.

In [ ]:
@rt
def htmlredirect(app): return HtmxResponseHeaders(location="http://example.org")

`APIRouter` lets you organize routes across multiple files in a FastHTML app, with an optional `prefix` parameter that prepends to all routes in that router (e.g., `prefix=/products` makes all routes start with `/products`).

```python
# products.py
ar = APIRouter(prefix="/products")

@ar("/all")
def all_products(req):
    return Div(
        "Welcome to the Products Page! Click the button below to look at the details for product 42",
        Div(
            Button("Details",hx_get=req.url_for("details", pid=42),hx_target="#products_list",hx_swap="outerHTML",),
        ),
        id="products_list",
    )

@ar.get("/", name="details")
def details(pid: int): return f"Here are the product details for ID: {pid}"
```

Adding the products' `APIRouter` to the `app`:

```python
# main.py
from products import ar

app, rt = fast_app()
ar.to_app(app)

@rt
def index():
    return Div(
        "Products",
        hx_get=ar.rt_funcs.all_products,
        hx_swap="outerHTML",
    )
```

Toasts can be of four types:

- info
- success
- warning
- error

Toasts require the use of the `setup_toasts()` function, plus every handler needs:

- The session argument
- Must return FT components

```python
setup_toasts(app)

@rt
def toasting(session):
    add_toast(session, f"cooked", "info")
    add_toast(session, f"ready", "success")
    return Titled("toaster")
```

`setup_toasts(duration)` allows you to specify how long a toast will be visible before disappearing.10 seconds.

Authentication and authorization are handled with Beforeware, which functions that run before the route handler is called.

In [ ]:
def user_auth_before(req, sess):
    # `auth` key in the request scope is automatically provided to any handler which requests it and can not be injected
    auth = req.scope['auth'] = sess.get('auth', None)
    if not auth: return RedirectResponse('/login', status_code=303)

beforeware = Beforeware(
    user_auth_before,
    skip=[r'/favicon\.ico', r'/static/.*', r'.*\.css', r'.*\.js', '/login', '/']
)

app, rt = fast_app(before=beforeware)

FastHTML introduces several tools for working with SSE which are covered in the example below. While concise, there's a lot going on in this function so we've annotated it quite a bit.

In [ ]:
import random

FastHTML supports the HTMX SSE extension.

In [ ]:
hdrs=(Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),)
app,rt = fast_app(hdrs=hdrs)

@rt
def index(): return Div(hx_ext="sse", sse_connect="/numstream", hx_swap="beforeend show:bottom", sse_swap="message")

# `signal_shutdown()` gets an event that is set on shutdown
shutdown_event = signal_shutdown()

async def number_generator():
    while not shutdown_event.is_set():
        data = Article(random.randint(1, 100))
        yield sse_message(data)

@rt
async def numstream(): return EventStream(number_generator())

FastHTML provides useful tools for HTMX's websockets extension.

In [ ]:
app, rt = fast_app(exts='ws')

def mk_inp(): return Input(id='msg', autofocus=True)

@rt
async def index(request):
    # `ws_send` tells HTMX to send a message to the nearest websocket based on the trigger for the form element
    cts = Div(
        Div(id='notifications'),
        Form(mk_inp(), id='form', ws_send=True),
        hx_ext='ws', ws_connect='/ws')
    return Titled('Websocket Test', cts)

async def on_connect(send): await send(Div('Hello, you have connected', id="notifications"))
async def on_disconnect(ws): print('Disconnected!')

@app.ws('/ws', conn=on_connect, disconn=on_disconnect)
async def ws(msg:str, send):
    # websocket hander returns/sends are treated as OOB swaps
    await send(Div('Hello ' + msg, id="notifications"))
    return Div('Goodbye ' + msg, id="notifications"), mk_inp()

### Single File Uploads

`Form` defaults to "multipart/form-data". A Starlette UploadFile is passed to the handler.

```python
upload_dir = Path("filez")

@rt
def index():
    return (
        Form(hx_post=upload, hx_target="#result")(
            Input(type="file", name="file"),
            Button("Upload", type="submit")),
        Div(id="result")
    )

@rt
async def upload(file: UploadFile):
    filebuffer = await file.read()
    (upload_dir / file.filename).write_bytes(filebuffer)
    return P('Size: ', file.size)
```

For multi-file, use `Input(..., multiple=True)`, and a type annotation of `list[UploadFile]` in the handler. 

## Fastlite 

Fastlite and the MiniDataAPI specification it's built on are a CRUD-oriented API for working with SQLite. APSW and apswutils is used to connect to SQLite, optimized for speed and clean error handling.

In [ ]:
from fastlite import *

In [ ]:
db = database(':memory:') # or database('data/app.db')

Tables are normally constructed with classes, field types are specified as type hints. 

In [ ]:
class Book: isbn: str; title: str; pages: int; userid: int
# The transform arg instructs fastlite to change the db schema when fields change.
# Create only creates a table if the table doesn't exist.
books = db.create(Book, pk='isbn', transform=True)
                
class User: id: int; name: str; active: bool = True
# If no pk is provided, id is used as the primary key.
users = db.create(User, transform=True)

### Fastlite CRUD operations

Every operation in fastlite returns a full superset of dataclass functionality. 

Creating records:

In [ ]:
user = users.insert(name='Alex',active=False)
user

User(id=1, name='Alex', active=0)

Listing records:

In [ ]:
# List all records
users()
# Limit, offset, and order results:
users(order_by='name', limit=2, offset=1)
# Filter on the results
users(where="name='Alex'")
# Placeholder for avoiding injection attacks
users("name=?", where_args=('Alex',))


[User(id=1, name='Alex', active=0)]

Fetching a single record using the pk inside square brackets returns a flexiclass:

In [ ]:
users[user.id]

User(id=1, name='Alex', active=0)

Test if a record exists by using `in` keyword on primary key:

In [ ]:
1 in users

True

Updates return the updated record.

In [ ]:
user.name='Lauren'
user.active=True
users.update(user)

User(id=1, name='Lauren', active=1)

`.xtra()` to constrain queries:

In [ ]:
users.xtra(active=True)
users()

[User(id=1, name='Lauren', active=1)]

Deleting a record

In [ ]:
users.delete(user.id)

<Table user (id, name, active)>

NotFoundError is raised by square bracket searches, updates, and deletes.

In [ ]:
try: users['Amy']
except NotFoundError: print('User not found')

User not found


## MonsterUI

MonsterUI is a UI framework for FastHTML that lets you build beautiful web interfaces with minimal code. It combines the simplicity of Python with the power of Tailwind. Perfect for data scientists, ML engineers, and developers who want to quickly turn their Python code into polished web apps without the complexity of traditional UI frameworks. Follows semantic HTML patterns when possible.

Monster adds the Tailwind-based libraries FrankenUI and DaisyUI to FastHTML, as well as Python's mistle. A minimal app:

In [ ]:
from monsterui.all import *

app, rt = fast_app(hdrs=Theme.blue.headers()) # Use MonsterUI blue theme

@rt
def index():
    socials = (('github','https://github.com/AnswerDotAI/MonsterUI'),
               ('twitter','https://twitter.com/isaac_flath/'),
               ('linkedin','https://www.linkedin.com/in/isaacflath/'))
    return Titled("Your First App",
        Card(
            H1("Welcome!"),
            P("Your first MonsterUI app", cls=TextPresets.muted_sm),
            # DivLAigned and UkIconLink are non-semantic MonsterUI FT Components
            footer=DivLAligned(*[UkIconLink(icon,href=url) for icon,url in socials])))


MonsterUI components are based on Franken UI/shadcn, which uses Tailwind:

In [ ]:
Grid(
    Card(H4("First Card")),
    Card(H4("Second Card")))

```html
<div class="grid grid-cols-1 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-2 xl:grid-cols-2 gap-4">
  <div class="uk-card ">
    <div class="uk-card-body space-y-6">
      <h4 class="uk-h4 ">First Card</h4>
    </div>
  </div>
  <div class="uk-card ">
    <div class="uk-card-body space-y-6">
      <h4 class="uk-h4 ">Second Card</h4>
    </div>
  </div>
</div>

```

List of all MonsterUI FastTags:

In [ ]:
from monsterui import all as monster
import inspect
from IPython.display import Markdown, display

In [ ]:
def isft(o): return o[0].isupper()
def ftsig(o): 
    try: return f'- {o}{inspect.signature(getattr(monster, o))}'
    except TypeError: return ''

fts = L(dir(monster)).filter(isft).map(ftsig)
display(Markdown('\n'.join(fts)))

- AT(*values)
- Abbr(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), title: str = None, **kwargs) -> fastcore.xml.FT
- Address(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Alert(*c, cls='', **kwargs) -> fastcore.xml.FT
- AlertT(*values)
- Article(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ArticleMeta(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ArticleTitle(*c, cls=(), **kwargs) -> fastcore.xml.FT
- BackgroundT(*values)
- Blockquote(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Button(*c: Union[str, fastcore.xml.FT], cls: Union[str, enum.Enum] = <ButtonT.default: 'uk-btn-default'>, submit=True, **kwargs) -> fastcore.xml.FT
- ButtonT(*values)
- Caption(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Card(*c, header: Union[fastcore.xml.FT, Iterable[fastcore.xml.FT]] = None, footer: Union[fastcore.xml.FT, Iterable[fastcore.xml.FT]] = None, body_cls='space-y-6', header_cls=(), footer_cls=(), cls=(), **kwargs) -> fastcore.xml.FT
- CardBody(*c, cls=(), **kwargs) -> fastcore.xml.FT
- CardContainer(*c, cls=<CardT.default: 'uk-card-default'>, **kwargs) -> fastcore.xml.FT
- CardFooter(*c, cls=(), **kwargs) -> fastcore.xml.FT
- CardHeader(*c, cls=(), **kwargs) -> fastcore.xml.FT
- CardT(*values)
- CardTitle(*c, cls=(), **kwargs)
- Center(*c, vertical: bool = True, horizontal: bool = True, cls=(), **kwargs) -> fastcore.xml.FT
- CheckboxX(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Cite(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- CodeBlock(*c: str, cls: enum.Enum | str | tuple = (), code_cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- CodeSpan(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Container(*c, cls=('mt-5', <ContainerT.xl: 'uk-container-xl'>), **kwargs) -> fastcore.xml.FT
- ContainerT(*values)
- Data(*c: fastcore.xml.FT | str, value: str = None, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Del(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Details(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Dfn(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- DiceBearAvatar(seed_name: str, h: int = 20, w: int = 20)
- DivCentered(*c, cls='space-y-4', vstack=True, **kwargs) -> fastcore.xml.FT
- DivFullySpaced(*c, cls='w-full', **kwargs)
- DivHStacked(*c, cls='space-x-4', **kwargs) -> fastcore.xml.FT
- DivLAligned(*c, cls='space-x-4', **kwargs) -> fastcore.xml.FT
- DivRAligned(*c, cls='space-x-4', **kwargs) -> fastcore.xml.FT
- DivVStacked(*c, cls='space-y-4', **kwargs) -> fastcore.xml.FT
- Divider(*c, cls=('my-4', <DividerT.icon: 'uk-divider-icon'>), **kwargs) -> fastcore.xml.FT
- DividerLine(lwidth=2, y_space=4)
- DividerSplit(*c, cls=(), line_cls=(), text_cls=())
- DividerT(*values)
- DropDownNavContainer(*li, cls=<NavT.primary: 'uk-nav-primary'>, parent=True, uk_nav=False, uk_dropdown=True, **kwargs) -> fastcore.xml.FT
- Em(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- FastHTML(*args, pico=False, debug=False, routes=None, middleware=None, title: str = 'FastHTML page', exception_handlers=None, on_startup=None, on_shutdown=None, lifespan=None, hdrs=None, ftrs=None, exts=None, before=None, after=None, surreal=True, htmx=True, default_hdrs=True, sess_cls=<class 'starlette.middleware.sessions.SessionMiddleware'>, secret_key=None, session_cookie='session_', max_age=31536000, sess_path='/', same_site='lax', sess_https_only=False, sess_domain=None, key_fname='.sesskey', body_wrap=<function noop_body>, htmlkw=None, nb_hdrs=False)
- Fieldset(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Figure(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- FlexT(*values)
- Form(*c, cls='space-y-3', **kwargs) -> fastcore.xml.FT
- FormLabel(*c, cls=(), **kwargs) -> fastcore.xml.FT
- GenericLabelInput(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', container=functools.partial(<function ft>, 'div', void_=False), cls='', id=None, input_fn=<function noop>, **kwargs)
- Grid(*div, cols_min: int = 1, cols_max: int = 4, cols_sm: int = None, cols_md: int = None, cols_lg: int = None, cols_xl: int = None, cols: int = None, cls='gap-4', **kwargs) -> fastcore.xml.FT
- H1(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- H2(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- H3(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- H4(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- H5(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- H6(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT

- I(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Input(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Ins(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Kbd(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Label(*c, cls=(), **kwargs) -> fastcore.xml.FT
- LabelCheckboxX(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', container=functools.partial(<function ft>, 'div', void_=False), cls='flex items-center space-x-2', id='', **kwargs) -> fastcore.xml.FT
- LabelInput(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', cls='space-y-2', id='', **kwargs) -> fastcore.xml.FT
- LabelRadio(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', container=functools.partial(<function ft>, 'div', void_=False), cls='flex items-center space-x-2', id='', **kwargs) -> fastcore.xml.FT
- LabelRange(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', cls='space-y-6', id='', value='', min=None, max=None, step=None, label_range=True, *, container=functools.partial(<function ft>, 'div', void_=False)) -> fastcore.xml.FT
- LabelSelect(*option, label: str | fastcore.xml.FT, lbl_cls='', input_cls='', container=functools.partial(<function ft>, 'div', void_=False), cls='space-y-2', id='', **kwargs)
- LabelSwitch(label: str | fastcore.xml.FT, lbl_cls='', input_cls='', cls='space-x-2', id='', *, container=functools.partial(<function ft>, 'div', void_=False)) -> fastcore.xml.FT
- LabelT(*values)
- LabelTextArea(label: str | fastcore.xml.FT, value='', lbl_cls='', input_cls='', cls='space-y-2', id='', **kwargs) -> fastcore.xml.FT
- Legend(*c, cls=(), **kwargs) -> fastcore.xml.FT
- LiStep(*c, cls='', data_content=None, **kwargs) -> fastcore.xml.FT
- ListT(*values)
- Loading(cls=(<LoadingT.bars: 'loading-bars'>, <LoadingT.lg: 'loading-large'>), htmx_indicator=False, **kwargs) -> fastcore.xml.FT
- LoadingT(*values)
- Mark(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Meter(*c: fastcore.xml.FT | str, value: float = None, min: float = None, max: float = None, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Modal(*c, header=None, footer=None, cls=(), dialog_cls=(), header_cls='p-6', body_cls='space-y-6', footer_cls=(), id='', open=False, **kwargs) -> fastcore.xml.FT
- ModalBody(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ModalCloseButton(*c, cls=(), htmx=False, **kwargs) -> fastcore.xml.FT
- ModalContainer(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ModalDialog(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ModalFooter(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ModalHeader(*c, cls=(), **kwargs) -> fastcore.xml.FT
- ModalTitle(*c, cls=(), **kwargs) -> fastcore.xml.FT
- NavBar(*c, brand=h3(('Title',),{'class': 'uk-h3 '}), right_cls='items-center space-x-4', mobile_cls='space-y-4', sticky: bool = False, uk_scrollspy_nav: bool | str = False, cls='p-4', scrollspy_cls=<ScrollspyT.underline: 'navbar-underline'>, menu_id=None) -> fastcore.xml.FT
- NavCloseLi(*c, cls=(), **kwargs) -> fastcore.xml.FT
- NavContainer(*li, cls=<NavT.primary: 'uk-nav-primary'>, parent=True, uk_nav=False, uk_scrollspy_nav=False, sticky=False, **kwargs) -> fastcore.xml.FT
- NavDividerLi(*c, cls=(), **kwargs) -> fastcore.xml.FT
- NavHeaderLi(*c, cls=(), **kwargs) -> fastcore.xml.FT
- NavParentLi(*nav_container, cls=(), **kwargs) -> fastcore.xml.FT
- NavSubtitle(*c, cls=<TextPresets.muted_sm: 'text-gray-500 dark:text-gray-200 text-sm'>, **kwargs) -> fastcore.xml.FT
- NavT(*values)
- Options(*c, selected_idx: int = None, disabled_idxs: set = None)
- Output(*c: fastcore.xml.FT | str, form: str = None, for_: str = None, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- PicSumImg(h: int = 200, w: int = 200, id: int = None, grayscale: bool = False, blur: int = None, **kwargs) -> fastcore.xml.FT
- Placeholder(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Progress(*c, cls=(), value='', max='100', **kwargs) -> fastcore.xml.FT
- Q(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Radio(*c, cls=(), **kwargs) -> fastcore.xml.FT
- Range(*c, value='', label=True, min=None, max=None, step=None, cls=(), **kwargs) -> fastcore.xml.FT
- S(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Samp(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- ScrollspyT(*values)
- Section(*c, cls=(), **kwargs) -> fastcore.xml.FT
- SectionT(*values)
- Select(*option, inp_cls=(), cls=('h-10',), cls_custom='button: uk-input-fake dropdown: w-full', id='', name='', placeholder='', searchable=False, select_kwargs=None, **kwargs)
- Slider(*c, cls='', items_cls='gap-4', nav=True, nav_cls='', **kwargs) -> fastcore.xml.FT
- SliderContainer(*c, cls='', uk_slider=True, **kwargs) -> fastcore.xml.FT
- SliderItems(*c, cls='', **kwargs) -> fastcore.xml.FT
- SliderNav(cls='uk-position-small uk-hidden-hover', prev_cls='absolute left-0 top-1/2 -translate-y-1/2', next_cls='absolute right-0 top-1/2 -translate-y-1/2', **kwargs) -> fastcore.xml.FT
- Small(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- StepT(*values)
- Steps(*li, cls='', **kwargs) -> fastcore.xml.FT
- StepsT(*values)
- Strong(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Sub(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Subtitle(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = 'mt-1.5', **kwargs) -> fastcore.xml.FT
- Summary(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Sup(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- Switch(*c, cls=(), **kwargs) -> fastcore.xml.FT
- TabContainer(*li, cls='', alt=False, **kwargs) -> fastcore.xml.FT
- Table(*c, cls=(<TableT.middle: 'uk-table-middle'>, <TableT.divider: 'uk-table-divider'>, <TableT.hover: 'uk-table-hover'>, <TableT.sm: 'uk-table-sm'>), **kwargs) -> fastcore.xml.FT
- TableFromDicts(header_data: Sequence, body_data: Sequence[dict], footer_data=None, header_cell_render=<function Th>, body_cell_render=<function <lambda>>, footer_cell_render=<function <lambda>>, cls=(<TableT.middle: 'uk-table-middle'>, <TableT.divider: 'uk-table-divider'>, <TableT.hover: 'uk-table-hover'>, <TableT.sm: 'uk-table-sm'>), sortable=False, **kwargs) -> fastcore.xml.FT
- TableFromLists(header_data: Sequence, body_data: Sequence[Sequence], footer_data=None, header_cell_render=<function Th>, body_cell_render=<function Td>, footer_cell_render=<function Td>, cls=(<TableT.middle: 'uk-table-middle'>, <TableT.divider: 'uk-table-divider'>, <TableT.hover: 'uk-table-hover'>, <TableT.sm: 'uk-table-sm'>), sortable=False, **kwargs) -> fastcore.xml.FT
- TableT(*values)
- Tbody(*rows, cls=(), sortable=False, **kwargs)
- Td(*c, cls=(), shrink=False, expand=False, small=False)
- TextArea(*c, cls=(), **kwargs) -> fastcore.xml.FT
- TextPresets(*values)
- TextT(*values)
- Th(*c, cls=(), shrink=False, expand=False, small=False)
- Theme(*values)
- ThemeFont()
- ThemePicker(color=True, radii=True, shadows=True, font=True, mode=True, cls='p-4')
- ThemeRadii(*values)
- ThemeShadows()
- Time(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), datetime: str = None, **kwargs) -> fastcore.xml.FT
- Titled(title: str = 'FastHTML app', *c, cls=<ContainerT.xl: 'uk-container-xl'>, **kwargs) -> fastcore.xml.FT
- Toast(*c, cls='', alert_cls='', **kwargs) -> fastcore.xml.FT
- ToastHT(*values)
- ToastVT(*values)
- U(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT
- UkFormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6)
- UkIcon(icon: str, height: int = None, width: int = None, stroke_width: int = None, cls=(), **kwargs) -> fastcore.xml.FT
- UkIconLink(icon: str, height: int = None, width: int = None, stroke_width: int = None, cls=(), button: bool = False, **kwargs) -> fastcore.xml.FT
- Upload(*c, cls=(), multiple=False, accept=None, button_cls=<ButtonT.default: 'uk-btn-default'>, id=None, name=None, **kwargs) -> fastcore.xml.FT
- UploadZone(*c, cls=(), multiple=False, accept=None, id=None, name=None, **kwargs) -> fastcore.xml.FT
- Var(*c: fastcore.xml.FT | str, cls: enum.Enum | str | tuple = (), **kwargs) -> fastcore.xml.FT